# Project Notes (Sanitized for Git)

This repository contains a **sanitized** version of the Gracity Insects YOLOv8 Classification notebooks.
All tenant-specific identifiers (bucket names, namespaces, OCIDs, local absolute paths) have been replaced by placeholders.

**Author:** Cristina Varas Menadas  
**Last updated:** 2026-02-19

> To run these notebooks, set the configuration values in the first "Configuration" section of each notebook.


# Gracity Insects — 05. Export & Upload Artifacts

- Export best model to ONNX
- Upload run artifacts to Object Storage (`yolo/runs/...`)


## Configuration

Update these variables for your tenancy/project.

- **Bucket**: `<BUCKET_NAME>`
- **Dataset prefix** (images): `<PROJECT_PREFIX>/v1/raw/datasets/insects_kaggle_v1/`
- **Labels prefix** (metadata/manifests): `<PROJECT_PREFIX>/v1/labels/insects_kaggle_v1/`
- **Runs prefix** (artifacts): `<PROJECT_PREFIX>/yolo/runs/insects_kaggle_v1/`

We intentionally keep **`test/` as validation** for this starter project (to match your current bucket structure).

## 5.1 Imports

In [ ]:
from __future__ import annotations

from pathlib import Path
from typing import List

import oci
from oci.object_storage import ObjectStorageClient
from ultralytics import YOLO

## 5.2 Paths & OCI config

In [ ]:
RUN_DIR: str = "./runs"
RUN_NAME: str = ""  # <-- set run name
RUN_PATH = Path(RUN_DIR) / RUN_NAME
BEST_PT = RUN_PATH / "weights" / "best.pt"
assert BEST_PT.exists(), BEST_PT

BUCKET_NAME: str = "<BUCKET_NAME>"
RUNS_PREFIX: str = "<PROJECT_PREFIX>/yolo/runs/insects_kaggle_v1"

signer = oci.auth.signers.get_resource_principals_signer()
os_client = ObjectStorageClient(config={}, signer=signer)
namespace: str = os_client.get_namespace().data
print("Namespace:", namespace)

## 5.3 Export to ONNX

In [ ]:
model = YOLO(str(BEST_PT))
onnx_path = model.export(format="onnx", opset=17, dynamic=False)
print("Exported:", onnx_path)

## 5.4 Upload artifacts

In [ ]:
def upload_dir(local_dir: Path, dest_prefix: str) -> int:
    n: int = 0
    for p in local_dir.rglob("*"):
        if not p.is_file():
            continue
        rel = p.relative_to(local_dir).as_posix()
        obj_name = f"{dest_prefix}/{rel}"
        with p.open("rb") as f:
            os_client.put_object(
                namespace_name=namespace,
                bucket_name=BUCKET_NAME,
                object_name=obj_name,
                put_object_body=f.read(),
                content_type="application/octet-stream",
            )
        n += 1
    return n

dest_prefix = f"{RUNS_PREFIX}/{RUN_NAME}"
uploaded = upload_dir(RUN_PATH, dest_prefix)
print(f"Uploaded {uploaded} files to: {dest_prefix}")